# AG2 - Swarm

This version:
- Removes all Streamlit UI elements
- Converts UI inputs to constants at the top of the file
- Maintains the same agent roles and swarm structure
- Uses the latest pattern for registering handoffs with explicit parameter naming
- Prints results to the console instead of displaying in the UI
- Includes proper error handling with traceback<br>

You'll need to replace "your-api-key-here" with your actual API key and adjust any other constants as needed.

## SETUP

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

llm_config = {
    "api_type": "openai",
    "model": "gpt-4o-mini",
    "api_key": os.environ["OPENAI_API_KEY"]
}

## IMPORTS

In [2]:
from autogen import AssistantAgent, UserProxyAgent
from autogen.agentchat.contrib.swarm_agent import (
    SwarmResult,
    register_hand_off,
    OnCondition,
    AfterWork,
    AfterWorkOption,
    initiate_swarm_chat
)

## Constants

In [3]:
# Constants (previously UI settings)
API_KEY = os.environ["OPENAI_API_KEY"]
MODEL = "gpt-4o-mini"
TEMPERATURE = 0.7
RECIPE_IDEA = "Healthy vegetarian pasta dish with seasonal vegetables"
CUISINE_TYPE = "Italian"
DIETARY_RESTRICTIONS = ["Vegetarian"]
COOKING_TIME = 30

## Context variables

In [4]:
# Context variables
context = {
    "recipe_idea": RECIPE_IDEA,
    "cuisine_type": CUISINE_TYPE,
    "dietary_restrictions": DIETARY_RESTRICTIONS,
    "cooking_time": COOKING_TIME,
    "planner_response": "",
    "chef_response": "",
    "nutritionist_response": "",
    "final_recipe": ""
}

## Configure LLM

In [5]:
# Configure LLM
llm_config = {
    "api_type": "openai",
    "model": MODEL,
    "temperature": TEMPERATURE,
    "api_key": API_KEY
}

## Functions to record response

In [6]:
# Function to record planner's response
def record_planner_response(response: str, context_variables: dict) -> SwarmResult:
    """Record the recipe planner's response"""
    context_variables["planner_response"] = response
    return SwarmResult(context_variables=context_variables)

# Function to record chef's response
def record_chef_response(response: str, context_variables: dict) -> SwarmResult:
    """Record the chef's response"""
    context_variables["chef_response"] = response
    return SwarmResult(context_variables=context_variables)

# Function to record nutritionist's response and finalize recipe
def record_nutritionist_response(response: str, context_variables: dict) -> SwarmResult:
    """Record the nutritionist's response and create final recipe"""
    context_variables["nutritionist_response"] = response
    
    # Combine all responses into a final recipe
    final_recipe = f"""
    # {context_variables['recipe_idea']}
    
    ## Ingredients and Plan
    {context_variables['planner_response']}
    
    ## Cooking Instructions
    {context_variables['chef_response']}
    
    ## Nutritional Information
    {context_variables['nutritionist_response']}
    """
    
    context_variables["final_recipe"] = final_recipe
    return SwarmResult(context_variables=context_variables)

## Create Agent (Recipe Planner)

In [7]:
recipe_planner = AssistantAgent(
    name="RecipePlanner",
    system_message=f"""You are a Recipe Planner who specializes in planning recipes.
    Based on the user's idea, create a structured recipe plan with:
    1. A catchy title for the recipe
    2. A list of all required ingredients with quantities
    3. Any special equipment needed
    
    Consider these requirements:
    - Cuisine type: {CUISINE_TYPE}
    - Dietary restrictions: {', '.join(DIETARY_RESTRICTIONS) if DIETARY_RESTRICTIONS else 'None'}
    - Maximum cooking time: {COOKING_TIME} minutes
    
    Be thorough but concise. Focus only on planning, not cooking instructions.
    """,
    llm_config=llm_config,
    functions=[record_planner_response]
)

## Create Agent (Chef)

In [8]:
chef = AssistantAgent(
    name="Chef",
    system_message=f"""You are a professional Chef who creates detailed cooking instructions.
    Based on the Recipe Planner's ingredients and plan, create step-by-step cooking instructions that:
    1. Are clear and easy to follow
    2. Include cooking times and temperatures
    3. Mention techniques and tips for best results
    4. Can be completed within {COOKING_TIME} minutes total
    
    Be thorough but practical. Focus only on the cooking process.
    """,
    llm_config=llm_config,
    functions=[record_chef_response]
)

## Create Agent (Nutritionist)

In [9]:
nutritionist = AssistantAgent(
    name="Nutritionist",
    system_message=f"""You are a Nutritionist who provides health information about recipes.
    Based on the recipe plan and cooking instructions, provide:
    1. Estimated nutritional information (calories, protein, carbs, fats)
    2. Health benefits of key ingredients
    3. Suggestions for healthy substitutions if applicable
    4. Serving suggestions and portion advice
    
    Consider these dietary restrictions: {', '.join(DIETARY_RESTRICTIONS) if DIETARY_RESTRICTIONS else 'None'}
    Be informative but concise. Focus only on nutritional aspects.
    """,
    llm_config=llm_config,
    functions=[record_nutritionist_response]
)

## Create User Agent (NOT USED)

In [10]:
user_proxy = UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    code_execution_config=False
)

## Agent functions and handoffs (agent workflow)
- Register functions with agents
- Register handoffs

**register_hand_off**
- https://docs.ag2.ai/docs/api-reference/autogen/agentchat/register_hand_off
```python
register_hand_off(
    agent: ConversableAgent,
    hand_to: list[OnCondition | OnContextCondition | AfterWork] | OnCondition | OnContextCondition | AfterWork
) -> None
```

**OnCondition**
- https://docs.ag2.ai/docs/api-reference/autogen/OnCondition
```python
OnCondition(
    target: ConversableAgent | dict[str, Any] | None = None,
    condition: str | ContextStr | Callable[[ConversableAgent, list[dict[str, Any]]], str] | None = None,
    available: Callable[[ConversableAgent, list[dict[str, Any]]], bool] | str | ContextExpression | None = None
)
```

**AfterWork**
- https://docs.ag2.ai/docs/api-reference/autogen/AfterWork
```python
AfterWork(
    agent: AfterWorkOption | ConversableAgent | str | Callable[..., Any],
    next_agent_selection_msg: str | ContextStr | Callable[[ConversableAgent, list[dict[str, Any]]], str] | None = None)
```

### Handoff (Recipe Planner)

In [11]:
register_hand_off(
    agent=recipe_planner,
    hand_to=[
        OnCondition(
            target=chef,
            condition=lambda agent, context: "Create cooking instructions based on this plan."
        ),
        AfterWork(agent=recipe_planner)
    ]
)

### Handoff (Chef)

In [12]:
register_hand_off(
    agent=chef,
    hand_to=[
        OnCondition(
            target=nutritionist,
            condition=lambda agent, context: "Provide nutritional information for this recipe."
        ),
        AfterWork(agent=chef)
    ]
)

### Handoff (Nutritionist)

In [13]:
register_hand_off(
    agent=nutritionist,
    hand_to=[
        AfterWork(agent=nutritionist),
        AfterWork(AfterWorkOption.TERMINATE)
    ]
)

## Recipe creation swarm

In [14]:
prompt = f"""Create a recipe for: {RECIPE_IDEA}
Cuisine type: {CUISINE_TYPE}
Dietary restrictions: {', '.join(DIETARY_RESTRICTIONS) if DIETARY_RESTRICTIONS else 'None'}
Maximum cooking time: {COOKING_TIME} minutes
"""

print(prompt)

Create a recipe for: Healthy vegetarian pasta dish with seasonal vegetables
Cuisine type: Italian
Dietary restrictions: Vegetarian
Maximum cooking time: 30 minutes



In [15]:
try:
    chat_result, updated_context, last_agent = initiate_swarm_chat(
        initial_agent=recipe_planner,
        agents=[recipe_planner, chef, nutritionist],
        messages=prompt,
        context_variables=context,
    )
    
    print("Recipe created successfully!")

    # CHECK
    # print(type(recipe_planner))
    # print(type(chef))
    # print(type(nutritionist))
    # print(type(context))
    # print(len(context))
    # print(context.keys())
    # print(context.keys())
    # print(context)
    
    # Print the final recipe
    # print("\n" + "="*50)
    # print("FINAL RECIPE:")
    # print("="*50)
    # print(updated_context['final_recipe'])
    # print("="*50)
    
    # Print agent contributions
    # print("\nAGENT CONTRIBUTIONS:")
    # print("\nRecipe Planner:")
    # print(updated_context['planner_response'])
    
    # print("\nChef:")
    # print(updated_context['chef_response'])
    
    # print("\nNutritionist:")
    # print(updated_context['nutritionist_response'])
    
    # # Print chat history
    # print("\nFULL AGENT CONVERSATION:")
    # for msg in chat_result.chat_history:
    #     if msg["role"] == "assistant":
    #         print(f"\n{msg.get('name', 'Assistant')}: {msg['content']}")
    #     else:
    #         print(f"\n{msg['role'].title()}: {msg['content']}")
    #     print("-"*50)
            
except Exception as e:
    print(f"An error occurred: {str(e)}")
    import traceback
    traceback.print_exc()

_User (to chat_manager):

Create a recipe for: Healthy vegetarian pasta dish with seasonal vegetables
Cuisine type: Italian
Dietary restrictions: Vegetarian
Maximum cooking time: 30 minutes


--------------------------------------------------------------------------------

Next speaker: RecipePlanner



C:\Users\Pavel\projects\ai-llm-agents\.venv\Lib\site-packages\pydantic\main.py:1630: RuntimeWarning: fields may not start with an underscore, ignoring "_x"
  warnings.warn(f'fields may not start with an underscore, ignoring "{f_name}"', RuntimeWarning)


RecipePlanner (to chat_manager):

***** Suggested tool call (call_CgU4TAs0WAzhhW5SN6988y4J): record_planner_response *****
Arguments: 
{"response": "**Recipe Title:** Seasonal Veggie Delight Pasta\n\n**Ingredients:**\n- 8 oz whole wheat pasta (spaghetti or penne)\n- 1 cup cherry tomatoes, halved\n- 1 cup zucchini, diced\n- 1 cup bell peppers, diced (mix of colors)\n- 1 cup spinach leaves\n- 2 cloves garlic, minced\n- 2 tablespoons olive oil\n- 1 teaspoon dried oregano\n- 1 teaspoon salt\n- 1/2 teaspoon black pepper\n- 1/4 cup grated Parmesan cheese (optional for serving)\n- Fresh basil leaves for garnish (optional)\n\n**Special Equipment Needed:**\n- Large pot for boiling pasta\n- Skillet or frying pan\n- Colander for draining pasta\n- Cooking spoon"}
****************************************************************************************
***** Suggested tool call (call_2WPOTHAYTOtJq9yd8sdOlVZV): transfer_RecipePlanner_to_Chef *****
Arguments: 
{}
**************************************

C:\Users\Pavel\projects\ai-llm-agents\.venv\Lib\site-packages\autogen\agentchat\conversable_agent.py:3002: UserWarning: Function 'transfer_Chef_to_Nutritionist' is being overridden.
  warnings.warn(f"Function '{tool_sig['function']['name']}' is being overridden.", UserWarning)


Nutritionist (to chat_manager):

***** Suggested tool call (call_nJbdJU0vk6SSAchaS1oOPN56): record_nutritionist_response *****
Arguments: 
{"response": "**Estimated Nutritional Information (per serving, serves 4):**\n- Calories: ~300\n- Protein: ~12g\n- Carbohydrates: ~50g\n- Fats: ~10g\n\n**Health Benefits of Key Ingredients:**\n- **Whole Wheat Pasta:** High in fiber, which aids digestion and helps maintain a healthy weight. Provides complex carbohydrates for sustained energy.\n- **Zucchini:** Low in calories and packed with vitamins A and C. It helps with hydration and is a good source of antioxidants.\n- **Bell Peppers:** Rich in vitamins A and C, potassium, and fiber. They have anti-inflammatory properties and can boost your immune system.\n- **Spinach:** High in iron, calcium, and vitamins K and A. It supports bone health and provides antioxidants that protect against free radicals.\n- **Olive Oil:** Contains healthy monounsaturated fats and antioxidants, which are beneficial for 

## CHECK PRINT (updated_context)

In [16]:
print("*"*100)
print(type(updated_context))
print(len(updated_context))
print(updated_context.keys())
print(updated_context.values())

print("*"*100)
# print(updated_context['final_recipe'])  
print(f"[FINAL RECIPE]\n\n{updated_context['final_recipe']}")  # "FINAL RECIPE:"
print("*"*100)
print(f"[RECIPE PLANNER]\n\n{updated_context['planner_response']}")  # "AGENT CONTRIBUTIONS (Recipe Planner):"
print("*"*100)
print(f"[CHEF]\n\n{updated_context['chef_response']}")  # "AGENT CONTRIBUTIONS (Chef)"
print("*"*100)
print(f"[NUTRITIONIST]\n\n{updated_context['nutritionist_response']}")  # "AGENT CONTRIBUTIONS (Nutritionist)"

print("*"*100)

****************************************************************************************************
<class 'dict'>
8
dict_keys(['recipe_idea', 'cuisine_type', 'dietary_restrictions', 'cooking_time', 'planner_response', 'chef_response', 'nutritionist_response', 'final_recipe'])
dict_values(['Healthy vegetarian pasta dish with seasonal vegetables', 'Italian', ['Vegetarian'], 30, '**Recipe Title:** Seasonal Veggie Delight Pasta\n\n**Ingredients:**\n- 8 oz whole wheat pasta (spaghetti or penne)\n- 1 cup cherry tomatoes, halved\n- 1 cup zucchini, diced\n- 1 cup bell peppers, diced (mix of colors)\n- 1 cup spinach leaves\n- 2 cloves garlic, minced\n- 2 tablespoons olive oil\n- 1 teaspoon dried oregano\n- 1 teaspoon salt\n- 1/2 teaspoon black pepper\n- 1/4 cup grated Parmesan cheese (optional for serving)\n- Fresh basil leaves for garnish (optional)\n\n**Special Equipment Needed:**\n- Large pot for boiling pasta\n- Skillet or frying pan\n- Colander for draining pasta\n- Cooking spoon', '**Re

## CHECK PRINT (last_agent, chat_result)

In [25]:
print("*"*100)
print(last_agent)
print(type(chat_result))
print(type(chat_result.chat_history))
print(len(chat_result.chat_history))

print("*"*100)
for msg in chat_result.chat_history:  # "FULL AGENT CONVERSATION:"
    if msg["role"] == "assistant":
        print(f"[{msg.get('name', 'Assistant')}]: \n\n{msg['content']}")
    else:
        print(f"[{msg['role'].title()}]: \n\n{msg['content']}")
    print("*"*100)

****************************************************************************************************
Swarm agent --> Nutritionist
<class 'autogen.agentchat.chat.ChatResult'>
<class 'list'>
11
****************************************************************************************************
[Assistant]: 

Create a recipe for: Healthy vegetarian pasta dish with seasonal vegetables
Cuisine type: Italian
Dietary restrictions: Vegetarian
Maximum cooking time: 30 minutes

****************************************************************************************************
[RecipePlanner]: 

None
****************************************************************************************************
[Tool]: 


Swarm agent --> Chef
****************************************************************************************************
[User]: 

**Recipe Title:** Seasonal Veggie Delight Pasta

**Cooking Instructions:**

1. **Boil the Pasta:**
   - Fill a large pot with water and bring it to a boil ove

## GENERATED CODE by Augment

In [26]:
# from autogen import AssistantAgent, UserProxyAgent
# from autogen.agentchat.contrib.swarm_agent import (
#     SwarmResult,
#     register_hand_off,
#     OnCondition,
#     AfterWork,
#     AfterWorkOption,
#     initiate_swarm_chat
# )

# # Constants (previously UI settings)
# API_KEY = "your-api-key-here"  # Replace with your actual API key
# MODEL = "gpt-4"
# TEMPERATURE = 0.7
# RECIPE_IDEA = "Healthy vegetarian pasta dish with seasonal vegetables"
# CUISINE_TYPE = "Italian"
# DIETARY_RESTRICTIONS = ["Vegetarian"]
# COOKING_TIME = 30

# # Context variables
# context = {
#     "recipe_idea": RECIPE_IDEA,
#     "cuisine_type": CUISINE_TYPE,
#     "dietary_restrictions": DIETARY_RESTRICTIONS,
#     "cooking_time": COOKING_TIME,
#     "planner_response": "",
#     "chef_response": "",
#     "nutritionist_response": "",
#     "final_recipe": ""
# }

# # Function to record planner's response
# def record_planner_response(response: str, context_variables: dict) -> SwarmResult:
#     """Record the recipe planner's response"""
#     context_variables["planner_response"] = response
#     return SwarmResult(context_variables=context_variables)

# # Function to record chef's response
# def record_chef_response(response: str, context_variables: dict) -> SwarmResult:
#     """Record the chef's response"""
#     context_variables["chef_response"] = response
#     return SwarmResult(context_variables=context_variables)

# # Function to record nutritionist's response and finalize recipe
# def record_nutritionist_response(response: str, context_variables: dict) -> SwarmResult:
#     """Record the nutritionist's response and create final recipe"""
#     context_variables["nutritionist_response"] = response
    
#     # Combine all responses into a final recipe
#     final_recipe = f"""
#     # {context_variables['recipe_idea']}
    
#     ## Ingredients and Plan
#     {context_variables['planner_response']}
    
#     ## Cooking Instructions
#     {context_variables['chef_response']}
    
#     ## Nutritional Information
#     {context_variables['nutritionist_response']}
#     """
    
#     context_variables["final_recipe"] = final_recipe
#     return SwarmResult(context_variables=context_variables)

# def main():
#     # Configure LLM
#     llm_config = {
#         "api_type": "openai",
#         "model": MODEL,
#         "temperature": TEMPERATURE,
#         "api_key": API_KEY
#     }
    
#     # Create the agents
#     print("Creating agents...")
    
#     recipe_planner = AssistantAgent(
#         name="RecipePlanner",
#         system_message=f"""You are a Recipe Planner who specializes in planning recipes.
#         Based on the user's idea, create a structured recipe plan with:
#         1. A catchy title for the recipe
#         2. A list of all required ingredients with quantities
#         3. Any special equipment needed
        
#         Consider these requirements:
#         - Cuisine type: {CUISINE_TYPE}
#         - Dietary restrictions: {', '.join(DIETARY_RESTRICTIONS) if DIETARY_RESTRICTIONS else 'None'}
#         - Maximum cooking time: {COOKING_TIME} minutes
        
#         Be thorough but concise. Focus only on planning, not cooking instructions.
#         """,
#         llm_config=llm_config,
#     )
    
#     chef = AssistantAgent(
#         name="Chef",
#         system_message=f"""You are a professional Chef who creates detailed cooking instructions.
#         Based on the Recipe Planner's ingredients and plan, create step-by-step cooking instructions that:
#         1. Are clear and easy to follow
#         2. Include cooking times and temperatures
#         3. Mention techniques and tips for best results
#         4. Can be completed within {COOKING_TIME} minutes total
        
#         Be thorough but practical. Focus only on the cooking process.
#         """,
#         llm_config=llm_config,
#     )
    
#     nutritionist = AssistantAgent(
#         name="Nutritionist",
#         system_message=f"""You are a Nutritionist who provides health information about recipes.
#         Based on the recipe plan and cooking instructions, provide:
#         1. Estimated nutritional information (calories, protein, carbs, fats)
#         2. Health benefits of key ingredients
#         3. Suggestions for healthy substitutions if applicable
#         4. Serving suggestions and portion advice
        
#         Consider these dietary restrictions: {', '.join(DIETARY_RESTRICTIONS) if DIETARY_RESTRICTIONS else 'None'}
#         Be informative but concise. Focus only on nutritional aspects.
#         """,
#         llm_config=llm_config,
#     )
    
#     user_proxy = UserProxyAgent(
#         name="User",
#         human_input_mode="NEVER",
#         code_execution_config=False
#     )
    
#     # Register functions with agents
#     print("Setting up agent functions...")
    
#     # Register handoffs
#     print("Configuring agent workflow...")
    
#     register_hand_off(
#         recipe_planner,
#         [
#             OnCondition(
#                 target=chef,
#                 condition="Create cooking instructions based on this plan."
#             ),
#             AfterWork(record_planner_response, kwargs={"response": lambda: recipe_planner.last_message()["content"]})
#         ]
#     )
    
#     register_hand_off(
#         chef,
#         [
#             OnCondition(
#                 target=nutritionist,
#                 condition="Provide nutritional information for this recipe."
#             ),
#             AfterWork(record_chef_response, kwargs={"response": lambda: chef.last_message()["content"]})
#         ]
#     )
    
#     register_hand_off(
#         nutritionist,
#         [
#             AfterWork(record_nutritionist_response, kwargs={"response": lambda: nutritionist.last_message()["content"]}),
#             AfterWork(agent=AfterWorkOption.TERMINATE)
#         ]
#     )
    
#     # Start the swarm
#     print("Starting the recipe creation swarm...")
    
#     prompt = f"""Create a recipe for: {RECIPE_IDEA}
#     Cuisine type: {CUISINE_TYPE}
#     Dietary restrictions: {', '.join(DIETARY_RESTRICTIONS) if DIETARY_RESTRICTIONS else 'None'}
#     Maximum cooking time: {COOKING_TIME} minutes
#     """
    
#     try:
#         chat_result, updated_context, last_agent = initiate_swarm_chat(
#             initial_agent=recipe_planner,
#             agents=[recipe_planner, chef, nutritionist],
#             messages=prompt,
#             context_variables=context,
#         )
        
#         print("Recipe created successfully!")
        
#         # Print the final recipe
#         print("\n" + "="*50)
#         print("FINAL RECIPE:")
#         print("="*50)
#         print(updated_context['final_recipe'])
#         print("="*50)
        
#         # Print agent contributions
#         print("\nAGENT CONTRIBUTIONS:")
#         print("\nRecipe Planner:")
#         print(updated_context['planner_response'])
        
#         print("\nChef:")
#         print(updated_context['chef_response'])
        
#         print("\nNutritionist:")
#         print(updated_context['nutritionist_response'])
        
#         # Print chat history
#         print("\nFULL AGENT CONVERSATION:")
#         for msg in chat_result.chat_history:
#             if msg["role"] == "assistant":
#                 print(f"\n{msg.get('name', 'Assistant')}: {msg['content']}")
#             else:
#                 print(f"\n{msg['role'].title()}: {msg['content']}")
#             print("-"*50)
                
#     except Exception as e:
#         print(f"An error occurred: {str(e)}")
#         import traceback
#         traceback.print_exc()

# if __name__ == "__main__":
#     main()